## Download required data from yahoo finance

### download yahoo finance

In [3]:
%pip install yfinance

  Using cached yfinance-0.2.37-py2.py3-none-any.whl.metadata (11 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl.metadata (5.5 kB)
  Using cached lxml-5.1.0-cp311-cp311-win_amd64.whl.metadata (3.6 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached frozendict-2.4.0-py3-none-any.whl
  Using cached html5lib-1.1-py2.py3-none-any.whl.metadata (16 kB)
Using cached yfinance-0.2.37-py2.py3-none-any.whl (72 kB)
Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
Using cached lxml-5.1.0-cp311-cp311-win_amd64.whl (3.9 MB)
Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python311\\Scripts\\sample.exe' -> 'c:\\Python311\\Scripts\\sample.exe.deleteme'



### download stock data

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import date, timedelta, datetime
end_date = datetime.today().strftime('%Y-%m-%d')
start_date = '2018-01-01'

data = yf.download('NFLX', start = start_date, end = end_date)

[*********************100%%**********************]  1 of 1 completed


In [2]:
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,196.100006,201.649994,195.419998,201.070007,201.070007,10966900
2018-01-03,202.050003,206.210007,201.500000,205.050003,205.050003,8591400
2018-01-04,206.199997,207.050003,204.000000,205.630005,205.630005,6029600
2018-01-05,207.250000,210.020004,205.589996,209.990005,209.990005,7033200
2018-01-08,210.020004,212.500000,208.440002,212.050003,212.050003,5580200


In [ ]:
data.to_csv('netflix_stocks.csv')